In [1]:
import numpy as np
import pandas as pd
import gc
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
def reduce_mem_usage(df):
    """
    iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    @param df:
    @return:
    """
    start_mem = df.memory_usage().sum()
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
            df[col] = df[col].astype('str')

    end_mem = df.memory_usage().sum()
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
# test = reduce_mem_usage(pd.read_csv('../input/test.csv'))
# test.head()

In [4]:
# test.to_hdf('../input/test.h5', 'df')

In [5]:
reader = pd.read_csv('../input/train.csv', chunksize=5000000)
for i, chunk in enumerate(reader):
    chunk = reduce_mem_usage(chunk)
    gc.collect()
    print('chunk.shape: ', chunk.shape)
    chunk_1 = chunk[chunk['click'] == 1]
    print('chunk_1.shape: ', chunk_1.shape)
    chunk_0 = chunk[chunk['click'] == 0]
    print('chunk_0.shape: ', chunk_0.shape)
    del chunk
    gc.collect()
    chunk_1 = chunk_1.sample(int(0.1 * len(chunk_1)))
    chunk_0 = chunk_0.sample(len(chunk_1))
    print('Sampled: ')
    print('chunk_1.shape: ', chunk_1.shape)
    print('chunk_0.shape: ', chunk_0.shape)
    train = pd.concat([chunk_0, chunk_1])
    del chunk_0, chunk_1
    gc.collect()
    train.to_hdf('../input/train/train{}.h5'.format(i), 'df')
    del train
    gc.collect()

Memory usage of dataframe is 960000128.00 MB
Memory usage after optimization is: 515000128.00 MB
Decreased by 46.4%
chunk.shape:  (5000000, 24)
chunk_1.shape:  (869006, 24)
chunk_0.shape:  (4130994, 24)
Sampled: 
chunk_1.shape:  (86900, 24)
chunk_0.shape:  (86900, 24)
Memory usage of dataframe is 960000132.00 MB
Memory usage after optimization is: 515000132.00 MB
Decreased by 46.4%
chunk.shape:  (5000000, 24)
chunk_1.shape:  (792351, 24)
chunk_0.shape:  (4207649, 24)
Sampled: 
chunk_1.shape:  (79235, 24)
chunk_0.shape:  (79235, 24)
Memory usage of dataframe is 960000132.00 MB
Memory usage after optimization is: 515000132.00 MB
Decreased by 46.4%
chunk.shape:  (5000000, 24)
chunk_1.shape:  (899082, 24)
chunk_0.shape:  (4100918, 24)
Sampled: 
chunk_1.shape:  (89908, 24)
chunk_0.shape:  (89908, 24)
Memory usage of dataframe is 960000132.00 MB
Memory usage after optimization is: 515000132.00 MB
Decreased by 46.4%
chunk.shape:  (5000000, 24)
chunk_1.shape:  (892575, 24)
chunk_0.shape:  (410